In [1]:
from datasets import load_from_disk

ds_detected = load_from_disk("datasets\datasets_cutted\shoplifting_images_2")


c:\Users\chesn\miniconda3\envs\ml_py11_torchvision\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [83]:
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict, concatenate_datasets
import torchvision.transforms as T
import random


rotater1 = T.RandomRotation(degrees=(5,5))
rotater2 = T.RandomRotation(degrees=(-5,-5))
hor_flipper = T.RandomHorizontalFlip(p = 1)
inverter = T.RandomInvert(p = 1)
blurrer = T.GaussianBlur(kernel_size=(5, 9), sigma = 2)
grayscale = T.Grayscale(num_output_channels=3)


def get_adjusted_dataset(ds_detected):
    img_tansformers = [rotater1, rotater2, hor_flipper, inverter, blurrer, grayscale]
    dict_adjusted = {'image' : [], 'labels': []}
    for i in tqdm(range(ds_detected['train'].num_rows)):
        label = ds_detected['train'][i]['labels']
        img = ds_detected['train'][i]['image']
        for img_trans in img_tansformers:
            if(random.random() < 0.5 or img_trans == grayscale):
                dict_adjusted['image'].append(img_trans(img))
                dict_adjusted['labels'].append(label)

    return DatasetDict({
        'train': concatenate_datasets([ds_detected['test'], Dataset.from_dict(dict_adjusted)]),
        'test': ds_detected['test'],
    })

In [84]:
ds_adjusted = get_adjusted_dataset(ds_detected)

100%|██████████| 214/214 [00:03<00:00, 55.07it/s]


In [85]:
ds_adjusted

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 886
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 108
    })
})

In [87]:
ds_adjusted.save_to_disk("datasets\datasets_adjusted\shoplifting_images_2")

Saving the dataset (0/1 shards):   0%|          | 0/886 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 108/108 [00:00<00:00, 3179.51 examples/s]
